In [1]:
from Crypto.Util.number import long_to_bytes
import sympy

# Broken RSA

I tried to send you an important message with RSA, however I messed up my RSA implementation really badly. Can you still recover the flag?

------------------------------------------------------------------------------------------------------------------

The given public parameters are:

In [3]:
n = 27772857409875257529415990911214211975844307184430241451899407838750503024323367895540981606586709985980003435082116995888017731426634845808624796292507989171497629109450825818587383112280639037484593490692935998202437639626747133650990603333094513531505209954273004473567193235535061942991750932725808679249964667090723480397916715320876867803719301313440005075056481203859010490836599717523664197112053206745235908610484907715210436413015546671034478367679465233737115549451849810421017181842615880836253875862101545582922437858358265964489786463923280312860843031914516061327752183283528015684588796400861331354873
e = 16
c = 11303174761894431146735697569489134747234975144162172162401674567273034831391936916397234068346115459134602443963604063679379285919302225719050193590179240191429612072131629779948379821039610415099784351073443218911356328815458050694493726951231241096695626477586428880220528001269746547018741237131741255022371957489462380305100634600499204435763201371188769446054925748151987175656677342779043435047048130599123081581036362712208692748034620245590448762406543804069935873123161582756799517226666835316588896306926659321054276507714414876684738121421124177324568084533020088172040422767194971217814466953837590498718

In first place, we notice that the exponent is small and a power of two, i.e the message $m$ is such that

$$ m^{16} \equiv c \mod n.$$

Therefore, we are tempted to extract square roots, but we know that if n is of the form $n=pq$ with $p$ and $q$ 'safe primes' this problem is as hard as factoring $n$. So there must be another vulnerability.

Trying to factor $n$ is an idea, but first of all it's faster to check if it's worth a shot, i.e. check if:

In [4]:
sympy.isprime(n)

True

Lol... So, this RSA is not an RSA and, since $\phi(n)=n-1$ because $n$ is prime, we try in first place to find the inverse of $e \mod n-1$, but... 

In [5]:
sympy.gcd(e,n-1)

8

and so there is no such an inverse.

We have to follow the longer path: square rooting (nb: we know that computing square roots modulo a prime is doable 'fast' thanks to Tonelli--Shanks' algorithm). 

In [6]:
def legendre(a, p):
    return pow(a, (p - 1) // 2, p)


def tonelli(n, p):
    assert legendre(n, p) == 1, "not a square (mod p)"
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1
    if s == 1:
        return pow(n, (p + 1) // 4, p)
    for z in range(2, p):
        if p - 1 == legendre(z, p):
            break
    c = pow(z, q, p)
    r = pow(n, (q + 1) // 2, p)
    t = pow(n, q, p)
    m = s
    t2 = 0
    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 - 1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r

The problem is that there are 2 square roots modulo an odd prime, and so we have to check every possibility (luckily not many). So finally we write

In [9]:
r8 = []
r4 = []
r2 = []
m = []
x = tonelli(c,n)
r8.append(x)
r8.append(n-x)

for d8 in r8:
    if legendre(d8,n) == 1:
        x = tonelli(d8,n)
        r4.append(x)
        r4.append(n-x)

for d4 in r4:
    if legendre(d4,n) == 1:
        x = tonelli(d4,n)
        r2.append(x)
        r2.append(n-x)

for d2 in r2:
    if legendre(d2,n) == 1:
        x = tonelli(d2,n)
        m.append(x)
        m.append(n-x)
        
print(len(m))

8


We can than easilly check which one is the original message, obtaining:

In [10]:
print(long_to_bytes(m[1]).decode())

Hey, if you are reading this maybe I didn't mess up my code too much. Phew. I really should play more CryptoHack before rushing to code stuff from scratch again. Here's the flag: crypto{m0dul4r_squ4r3_r00t}
